In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from lohrasb.best_estimator import BaseModel
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, make_scorer
from xgboost import *

from lohrasb import logger


2023-07-03 19:08:35,570 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-03 19:08:35,575 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-03 19:08:35,578 :: matplotlib :: interactive is False
2023-07-03 19:08:35,579 :: matplotlib :: platform is darwin
2023-07-03 19:08:35,593 :: graphviz._tools :: deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-03 19:08:35,595 :: graphviz._tools :: deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-03 19:08:35,597 :: graphviz._tools :: deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2023-07-03 19:08:35,600 :: graphviz._tools :: deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2023-07-03 19:08:35,604 :: graphviz._tools :: deprecate positio

#### Example: Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]


X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define estimator and set its arguments 


In [6]:
estimator = XGBClassifier()
estimator_params = {
        "booster": ["gbtree","dart"],
        "eval_metric": ["auc"],
        "max_depth": [4, 5, 10],
        "gamma": [0.1, 1.2],
        "subsample": [0.8,0.9,1.0],

    }
    

In [7]:
kwargs = {  # params for fit method or fit_params 
            'fit_grid_kwargs' :{
            'sample_weight':None,
            },
            # params for GridSearchCV 
            'grid_search_kwargs' : {
            'estimator':estimator,
            'param_grid':estimator_params,
            'scoring' :'f1',
            'verbose':3,
            'n_jobs':-1,
            'cv':KFold(2),
            }
            }


In [8]:

obj = BaseModel().optimize_by_gridsearchcv(
        kwargs=kwargs    
        )

#### Build sklearn pipeline

In [9]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [10]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


Fitting 2 folds for each of 36 candidates, totalling 72 fits
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=1.0;, score=0.700 total time=   1.5s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=1.0;, score=0.703 total time=   1.5s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.8;, score=0.695 total time=   1.5s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.8;, score=0.701 total time=   1.6s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.9;, score=0.701 total time=   1.6s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.9;, score=0.698 total time=   1.6s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=5, subsample=0.8;, score=0.686 total time=   2.1s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=5, subsample=0.8;, score=0.692 total time=   2.2s
[CV

#### Check performance of the pipeline

In [11]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.7121748178980228
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.77      0.66      0.71      2588

    accuracy                           0.87     10746
   macro avg       0.83      0.80      0.81     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7652  506]
 [ 877 1711]]


#### Part 2:  Use BestModel as a standalone estimator 

In [12]:
X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)

#### Transform features to make them ready for model input

In [13]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [14]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [15]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

Fitting 2 folds for each of 36 candidates, totalling 72 fits
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=1.0;, score=0.700 total time=   1.4s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=1.0;, score=0.703 total time=   1.4s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.9;, score=0.701 total time=   1.5s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.9;, score=0.698 total time=   1.5s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.8;, score=0.701 total time=   1.5s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.8;, score=0.695 total time=   1.5s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=5, subsample=0.8;, score=0.692 total time=   1.8s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=5, subsample=0.8;, score=0.686 total time=   1.8s
[CV

#### Check performance of the pipeline

In [16]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))

F1 score : 
0.7121748178980228
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.77      0.66      0.71      2588

    accuracy                           0.87     10746
   macro avg       0.83      0.80      0.81     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7652  506]
 [ 877 1711]]


In [17]:
obj.get_best_estimator()

XGBClassifier(base_score=0.5, booster='dart', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=1.2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [18]:
obj.best_estimator

XGBClassifier(base_score=0.5, booster='dart', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=1.2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

#### Get fitted grid search object and its attributes

In [19]:
GridSearchObj = obj.get_optimized_object()
GridSearchObj.cv_results_

{'mean_fit_time': array([ 1.49201202,  1.46559143,  1.40045798,  1.77267969,  1.71076941,
         1.64537251,  3.48095155,  3.43693662,  3.3223381 ,  1.39130998,
         1.37283218,  1.31752145,  1.71530735,  1.73173308,  1.70506215,
         3.69885099,  3.69985747,  3.45168602,  6.43921351,  6.58118749,
         6.56690753,  7.28286541,  7.37359297,  7.20344388, 11.08435142,
        10.84559107, 10.78654206,  6.38398695,  6.89477932,  7.00952816,
         8.22749186,  8.731552  ,  8.51080835, 11.89149344, 10.26948249,
         8.02026498]),
 'std_fit_time': array([0.00964117, 0.01053047, 0.00705779, 0.00116146, 0.00516057,
        0.01241362, 0.00576854, 0.00305557, 0.03398204, 0.01119709,
        0.00884116, 0.01091945, 0.00796449, 0.00427008, 0.01183105,
        0.00759995, 0.00846553, 0.02881801, 0.02022171, 0.05548358,
        0.00922453, 0.03435147, 0.00398195, 0.04388797, 0.00553262,
        0.04685402, 0.03126919, 0.00575089, 0.02035844, 0.00381398,
        0.05352592, 0.044